In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import MaxNLocator
import random
import csv
from datetime import datetime
import time

from langchain.schema.runnable import RunnableSequence

In [2]:
# Do imports for deh experiments specific modules
from pathlib import Path

utils_folder = Path("..")
sys.path.append(str(utils_folder))

utils_folder = Path("../src/deh")
sys.path.append(str(utils_folder))

utils_folder = Path(".")
sys.path.append(str(utils_folder))

import squad_scoring
import deh_prompts
import deh_vector_store
globals().update(deh_vector_store.__dict__)
import deh_squad_data
import deh_hyde
import deh_experiments_config
globals().update(deh_experiments_config.__dict__)
import deh_globals
globals().update(deh_globals.__dict__)
import deh_llm
globals().update(deh_llm.__dict__)

##### Loading SQuAD data

In [3]:
csv_file_path = f"{DATA_ROOT}/qas_with_contexts.csv"

print(f"Loading squad data...\n")

column_names = ["title", "squad_context", "qid", "question", "is_impossible", "answer"]
squad_raw = pd.read_csv(f"{DATA_ROOT}/squad_raw.csv", names=column_names, skiprows=1)
df_squad_raw = pd.DataFrame(squad_raw)
print(f"Number of raw entries in squad_raw: {len(df_squad_raw)}")

df_titles = pd.DataFrame(df_squad_raw['title'].unique(), columns=["title"])
print(f"Number of unique titles: {len(df_titles)}")

df_contexts = pd.DataFrame(df_squad_raw['squad_context'].unique(), columns=["squad_context"])
print(f"Number of unique contexts: {len(df_contexts)}")

df_qas = df_squad_raw[['title', 'squad_context', 'qid', 'question', 'is_impossible']].drop_duplicates()
df_qas = df_qas.reset_index(drop=True)
print(f"Number of unique questions: {len(df_qas)}")

df_squad_answers = df_squad_raw[['qid', 'question', 'answer']].drop_duplicates()
print(f"Number of unique answers: {len(df_squad_answers)}")           
            

Loading squad data...

Number of raw entries in squad_raw: 26232
Number of unique titles: 35
Number of unique contexts: 1204
Number of unique questions: 11858
Number of unique answers: 16209


##### Intialize the Vector Store (Chroma; Milvus not yet included); if configured, re-chunk the data

In [4]:
contexts = list(df_contexts["squad_context"].values)
if CHUNK_SQUAD_DATASET:    
    deh_vector_store.chunk_squad_dataset(contexts, dataset, CHUNK_SIZE, CHUNK_OVERLAP)
else:
    print("Chunking not foreseen. Skipping chunking.")

# Intiialize the Chroma vector store
vector_store = deh_vector_store.get_vector_store(DEFAULT_CHROMA_PREFIX, DEFAULT_CHUNKING_METHOD)

Chunking not foreseen. Skipping chunking.


##### Loading qas with contexts data (if data is not to be restored)

In [5]:
# If not restoring qas_with_contexts from the CSV file, then read
# the data from the csv file (i.e. it exists and is correct)
if not RESTORE_QAS_WITH_CONTEXTS:
    # Loading the question contexts from the CSV file
    print(f"Loading qas_with_contexts from the CSV file: {csv_file_path}")
    qas_with_contexts_csv_file_path = f"{DATA_ROOT}/qas_with_contexts.csv"

    df_qas_with_contexts = pd.read_csv(qas_with_contexts_csv_file_path) #, names=column_names)

    # drop the answer column if it exists, since it leads to duplicates
    if 'answer' in df_qas_with_contexts.columns:
        df_qas_with_contexts = df_qas_with_contexts.drop(columns=['answer'])

    df_qas_with_contexts = df_qas_with_contexts.drop_duplicates()
    print(f"Rows in dataframe df_qas_with_contexts: {len(df_qas_with_contexts)}")
    hyde_articles_cnt = df_qas_with_contexts['hyde_article'].notna().sum()
    hyde_based_contexts_cnt = df_qas_with_contexts['hyde_based_context'].notna().sum()
    print(f"Number of questions with Hyde articles: {hyde_articles_cnt}")
    print(f"Number of questions with Hyde based contexts: {hyde_based_contexts_cnt}")
else:
    print(f"RESTORE_QAS_WITH_CONTEXTS is True. Skipping loading qas_with_contexts from the CSV file.")



Loading qas_with_contexts from the CSV file: ../../../deh_data_results/data/qas_with_contexts.csv
Rows in dataframe df_qas_with_contexts: 11858
Number of questions with Hyde articles: 3918
Number of questions with Hyde based contexts: 3918


##### Restore qas with contexts (if configured); alternatively refreseh contexts in existing file

In [6]:
def restore_df_qas_with_contexts_file(csv_file_path, df_qas):

    # Add columns hyde_article, question_context, hyde_based_context to df_qas_with_contexts
    if not 'hyde_article' in df_qas.columns:
        df_qas['hyde_article'] = np.nan
    if not 'question_context' in df_qas.columns:
        df_qas['question_context'] = np.nan
    if not 'hyde_based_context' in df_qas.columns:
        df_qas['hyde_based_context'] = np.nan

    # Get Hyde data
    hyde_based_context_path = f"{HYDE_BASED_CONTEXTS_ROOT}/hyde_based_contexts.csv"
    df_hyde_based_contexts = pd.read_csv(hyde_based_context_path)

    print(f"Rows in dataframe df_hyde_based_contexts: {len(df_hyde_based_contexts)}")
    hyde_articles_cnt = df_hyde_based_contexts['hyde_article'].notna().sum()
    hyde_based_contexts_cnt = df_hyde_based_contexts['hyde_based_context'].notna().sum()

    # Merge df_qas with df_hyde_based_contexts based on the 'qid' column
    merged = df_qas.merge(df_hyde_based_contexts, on='qid', how='left', suffixes=('', '_df_hyde_based_contexts'))
    df_qas['hyde_article'] = merged['hyde_article_df_hyde_based_contexts']

    print(f"Number of questions with Hyde articles: {hyde_articles_cnt}")
    print(f"Number of questions with Hyde based contexts: {hyde_based_contexts_cnt}")

    df_qas.to_csv(csv_file_path, header=True, index=False)



In [7]:
# TODO an df_qas_with_contexts anpassen !!!!!!!!

if RESTORE_QAS_WITH_CONTEXTS:
    restore_df_qas_with_contexts_file(csv_file_path, df_qas)

# Refresh question contexts (normal contexts and hyde-based contexts)
# TODO: might be useful to first empty the two columns in the dataframe
if RESTORE_QAS_WITH_CONTEXTS or REFRESH_QUESTION_CONTEXTS or REFRESH_HYDE_CONTEXTS:
    print(f"Re-Generating contexts for the dataset and persisting the data...")
    list_of_qas = df_qas.to_dict(orient='records')

    for i, qa in enumerate(list_of_qas):
        # print(i)
        if i %100 == 0:
            print(f"Processing question {i}...")

        if RESTORE_QAS_WITH_CONTEXTS or REFRESH_QUESTION_CONTEXTS:
            question = qa["question"]
            
            top_docs = vector_store.similarity_search(
                query = question,
                k = VECTOR_STORE_TOP_K,
            )
            qa["question_context"] = "\n\n".join([top_doc.page_content for top_doc in top_docs])

        if RESTORE_QAS_WITH_CONTEXTS or REFRESH_HYDE_CONTEXTS:
            hyde_article = qa["hyde_article"]
            #print(f"hyde_article: {hyde_article}")
            if pd.isna(hyde_article):
                hyde_article = ""
            elif len(hyde_article) == 0:
                hyde_article = ""
            else:
                top_docs = vector_store.similarity_search(
                    query = hyde_article,
                    k = VECTOR_STORE_TOP_K,
                )

                qa["hyde_based_context"] = "\n\n".join([top_doc.page_content for top_doc in top_docs])

    df_qas = pd.DataFrame(list_of_qas)
    df_qas_with_contexts = df_qas.copy()
    df_qas.to_csv(csv_file_path, header=True, index=False)
else:
    print(f"Not restoring df_qas_with_contexts or refresh of the contexts...")

Not restoring df_qas_with_contexts or refresh of the contexts...


##### Show names of all dataframes

In [8]:
import copy 

print("Names of Dataframes and their lenghts:\n")
global_keys_copy = copy.deepcopy(list(globals().keys()))

my_l = [{n: len(globals()[n])} for n in global_keys_copy if n.startswith("df_")]
df_dfs = pd.DataFrame([(k, v) for d in my_l for k, v in d.items()], columns=['df name', 'rows'])
df_dfs


Names of Dataframes and their lenghts:



,df name,rows
0,df_experiments,7
1,df_squad_raw,26232
2,df_titles,35
3,df_contexts,1204
4,df_qas,11858
5,df_squad_answers,16209
6,df_qas_with_contexts,11858


##### Define functions that are needed for experiments

In [9]:
# Create the runnable chain
def get_runnable_chain(current_query_prompt, llm):
    runnable_chain = RunnableSequence(current_query_prompt | llm)
    return runnable_chain

In [10]:
# Get the Hyde context for a question
def get_hyde_based_context(question):
    hyde_based_context = df_qas_with_contexts[df_qas_with_contexts["question"] == question]
    if hyde_based_context.empty:
        return None
    else:
        return hyde_based_context["hyde_based_context"].values[0]
    

In [11]:
#%%capture

# Get the metrics for a set of predictions (preds) that have been generated in a run
def get_squad_metrics(dataset, preds, verbose=False):
    squad_metrics = squad_scoring.calc_squad_metrics(dataset, preds);
    return squad_metrics["precision"], squad_metrics["recall"], squad_metrics["f1"]


In [12]:
# Calculate the mean and confidence interval for a list of scores
# TODO: Check if this is calculation is correct !!
def calculate_mean_confidence_interval(scores_l):

    # Calculate mean
    mean = np.mean(scores_l)

    # Calculate 95% confidence interval
    sample_std_dev = np.std(scores_l, ddof=1)
    margin_of_error = 1.96 * sample_std_dev
    ci = (max(mean - margin_of_error, 0), min(mean + margin_of_error, 100))

    return mean, ci

In [13]:
# Generate a histogram for a list of scores and persist it
def generate_histogram(scores_l, mean, ci, results_folder_name, experiment_name):

    plt.clf
    plt.hist(scores_l, bins=30, density=False, edgecolor='black', alpha=0.6, color = 'lightblue' ) # color='aquamarine')
    plt.xlim(0, 100)

    plt.title(f"F1-Scores for {experiment_name} - (Bootstraps: {BOOTSTRAPS_N} - Sample Size: {SAMPLE_SIZE})", fontsize=10)
    plt.xlabel("F1-Score")
    plt.ylabel("Frequency")

    # Add a vertical line for the mean
    max_len = 6
    mean_label = f"{mean: .2f}".rjust(max_len)
    plt.axvline(mean, color='red', linestyle='dotted', linewidth=2, label=f'Mean F1:          {mean_label}')

    # Add vertical lines for the 95% confidence interval
    lower = f"{ci[0]: .2f}".rjust(max_len)
    upper = f"{ci[1]: .2f}".rjust(max_len)
    plt.axvline(ci[0], color='orange', linestyle='dashdot', linewidth=1.5, label=f"95% CI Lower:  {lower}")
    plt.axvline(ci[1], color='orange', linestyle='dashdot', linewidth=1.5, label=f"95% CI Upper:  {upper}")

    ax = plt.gca()  # Get current axis
    ax.xaxis.set_major_locator(MultipleLocator(10))  # Major ticks every 10 units
    ax.xaxis.set_minor_locator(MultipleLocator(5))   # Minor ticks every 5 units
    ax.yaxis.set_major_locator(MultipleLocator(10))   # Example: Major ticks every 5 units on the y-axis
    #ax.yaxis.set_major_formatter(PercentFormatter(xmax=len(data)))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=10))
    
    # Customize grid for major and minor ticks
    ax.grid(which='major', color='gray', linestyle='--', linewidth=0.5)
    ax.grid(which='minor', color='lightgray', linestyle=':', linewidth=0.5)

    # Add a legend
    plt.legend(prop={'family': 'monospace', 'size': 10})
    plt.legend(loc='upper right', fontsize=10)

    plt.savefig(os.path.join(results_folder_name, f"{experiment_name}_{BOOTSTRAPS_N}_{SAMPLE_SIZE}"))
    return plt

##### LLM as a Judge functions

In [14]:
def convert_string_to_answer_tuple(input_string):
    # Trim whitespace
    parts = input_string.strip()
    # Remove parentheses
    parts = parts.strip("()")
    # Split on comma
    parts = parts.split(", ")
    if len(parts) != 2:
        #raise ValueError("Invalid answer string")
        return ("NO", 0)

    answer = parts[0].strip().upper()  # "Yes" or "No"
    if answer not in ["YES", "NO"]:
        #raise ValueError("Invalid answer string")
        return ("NO", 0)
    try:
        score = float(parts[1])  # Convert score to a float
    except ValueError:
        return ("NO", 0)
    
    return (answer, score)

In [15]:
def get_majority_verdict(judge_verdicts):

    judge_answers = judge_verdicts[0]
    yes_count = judge_answers.count('YES')
    no_count = judge_answers.count('NO')

    if no_count > yes_count:
        return "NO", 0.0
    
    judge_scores = list(judge_verdicts[1])
    return "YES", sum(judge_scores) / yes_count

def get_majority_verditcs(all_judge_answers, all_judge_scores):

    majority_verdicts = []
    for per_question_judge_verdicts in zip(zip(*all_judge_answers), zip(*all_judge_scores)):
        majority_verdict = get_majority_verdict(per_question_judge_verdicts)
        majority_verdicts.append(majority_verdict)

    return majority_verdicts


In [16]:
def get_final_judges_verdicts(sample_ldict, experiment_name, hyde_context_needed=False):
    judge_current_query_prompt = deh_prompts.query_prompts[4]
    print(f"judge_current_query_prompt = {judge_current_query_prompt.template}\n")
    for ele in sample_ldict:
        print(ele)
        print("\n")

#    judge_llms = [MISTRAL_LATEST, GEMMA2_9B, QWEN2_5_7B]

    all_judge_answers = []
    all_judge_scores = []

    print("--------------------------------------------------------")
    for judge_i, judge_llm in enumerate(judge_llms):
        judge_chain = judge_current_query_prompt | get_llm(judge_current_query_prompt, True, judge_llm)
        judge_answers = []
        judge_scores = []

        for i, qa in enumerate(sample_ldict):

            if i % 10 == 0:
                print(f"Processing question {i} for judge {judge_i}...")

            question = qa["question"]
            context = qa["question_context"]
            if hyde_context_needed:
                hyde_based_context = qa["hyde_based_context"]
                context = "\n\n".join([context, hyde_based_context])
            answer_key = experiment_name.lower() + "_llm_answer"
            answer = qa[answer_key]

            # print(f"\nQuestion: {question}")
            # print(f"Context[:100]: {context[:100]}")
            # print(f"Answer: {answer}")

            response = judge_chain.invoke({"context": context, "question": question, "answer": answer})
            # print(f"Judge Response: {response.content}")
            a, s = convert_string_to_answer_tuple(response.content)
            judge_answers.append(a)
            judge_scores.append(s)
            
        all_judge_answers.append(judge_answers)
        all_judge_scores.append(judge_scores)

    final_judges_verdicts = get_majority_verditcs(all_judge_answers, all_judge_scores)
    return final_judges_verdicts

##### Define functions for Bootstrapping

In [17]:
# Get the current timestamp and format the start timestamp as a string
def get_timestamp_as_string():
    start_timestamp = datetime.now()
    start_timestamp_str = start_timestamp.strftime('%Y%m%d_%H%M%S')
    return start_timestamp_str

In [18]:
def calculate_scores(qas, query_prompt_idx, experiment_name, context_needed=False,
                     hyde_context_needed=False, suppress_answers=False):

    # Create the chain
    current_query_prompt = deh_prompts.query_prompts[query_prompt_idx]
    print(f"current_query_prompt = {current_query_prompt.template}\n")
    llm = get_llm(current_query_prompt)
    runnable_chain = get_runnable_chain(current_query_prompt, llm)

    # Generate the LLM answers for all questions and calculate per-answer metrics 
    # add each answer to the all_preds
    preds = {}
    all_preds= {}

    for i, qa in enumerate(qas):

        if i % 10 == 0:
            print(f"Processing question {i}...")

        qid = qa["qid"]
        question = qa["question"]

        if context_needed:
            if experiment_name in ["BASIC_RAG_HYDE"]:
                context = qa["hyde_based_context"]
            elif experiment_name in ["FULL_RAG"]:
                context = qa["question_context"] + "\n\n" + qa["hyde_based_context"]
            elif experiment_name in ["BASIC_RAG", "BASIC_RAG_DONT_LIE",
                                     "BASIC_RAG_SUPPRESS_ANSWERS", "BASIC_RAG_SEMANTIC"]:
                context = qa["question_context"]
            response = runnable_chain.invoke({"question": question, "context": context})
        else:
            response = runnable_chain.invoke({"question": question})

        llm_answer = response.content

        if "DONT KNOW" in llm_answer.upper() or llm_answer.upper().startswith("NO CONTEXT PROVIDED") or llm_answer.upper().startswith("NO INFORMATION GIVEN"):  
            llm_answer = ""

        preds[qid] = llm_answer
        all_preds[qid] = llm_answer

        scores = squad_scoring.calc_squad_metrics(dataset, preds)
        f1 = scores["f1"]
        precision = scores["precision"]
        recall = scores["recall"]

        preds = {}
        qa[f"{experiment_name.lower()}_llm_answer"] = llm_answer
        qa[f"{experiment_name.lower()}_f1"] = f1
        qa[f"{experiment_name.lower()}_precision"] = precision
        qa[f"{experiment_name.lower()}_recall"] = recall

    if suppress_answers:
        final_judges_verdicts = get_final_judges_verdicts(qas, experiment_name, hyde_context_needed)
        for i, qa in enumerate(qas):
            if final_judges_verdicts[i][0] == "NO":
                qa[f"{experiment_name.lower()}_llm_answer"] = ""
            else:
                if final_judges_verdicts[i][1] < JUDGES_SUPPRESS_THRESHOLD:
                    qa[f"{experiment_name.lower()}_llm_answer"] = ""

    return all_preds
        

In [19]:
def persist_results(results_folder_name, experiment_name, df):

    df.to_csv(f"{results_folder_name}/qas_{experiment_name.lower()}_scores.csv", header=True, index=False)
    df[['qid', 'question', f"{experiment_name.lower()}_llm_answer"]].to_csv(f"{results_folder_name}/qas_{experiment_name.lower()}_answers.csv", header=True, index=False)

In [20]:
def persist_experiments_config(results_folder_name, experiment_name, experiment_mean, experiment_ci, execution_time):

    params_str = f"Experiment: {experiment_name}\n"
    params_str = params_str + f"{datetime.now()}\n"
    params_str = params_str + f"Execution duration in minutes: {round(execution_time/60,2)}\n"
    params_str = params_str + "================================================\n\n"

    params_str = params_str + "--------------------------------------------------------\n"
    params_str = params_str + f"F1 mean: {experiment_mean}\n"
    params_str = params_str + f"95% CI ({experiment_ci[0]}, {experiment_ci[1]})\n\n"

    params_str = params_str + "--------------------------------------------------------\n"
    params_str = params_str + f"DATA_ROOT = {DATA_ROOT}\n"
    params_str = params_str + f"RESULTS_ROOT = {RESULTS_ROOT}\n"
    params_str = params_str + f"HYDE_BASED_CONTEXTS_ROOT  = {HYDE_BASED_CONTEXTS_ROOT}\n"
    params_str = params_str + f"data_file = {DATA_ROOT}/dev-v2.0.json\n\n"
    
    params_str = params_str + "--------------------------------------------------------\n"
    params_str = params_str + f"CHROMA_ROOT               = {CHROMA_ROOT}\n"
    params_str = params_str + f"VECTOR_STORE_TOP_K        = {VECTOR_STORE_TOP_K}\n"
    params_str = params_str + f"CHUNK_SIZE                = {CHUNK_SIZE}\n"
    params_str = params_str + f"CHUNK_OVERLAP             = {CHUNK_OVERLAP}\n"
    params_str = params_str + f"DEFAULT_CHROMA_PREFIX     = {DEFAULT_CHROMA_PREFIX}\n"
    params_str = params_str + f"DEFAULT_CHUNKING_METHOD   = {DEFAULT_CHUNKING_METHOD}\n"
    params_str = params_str + f"CHUNK_SQUAD_DATASET       = {CHUNK_SQUAD_DATASET}\n\n"
    
    params_str = params_str + "--------------------------------------------------------\n"
    params_str = params_str + f"REFRESH_QUESTION_CONTEXTS = {REFRESH_QUESTION_CONTEXTS}\n"
    params_str = params_str + f"REFRESH_HYDE_CONTEXTS     = {REFRESH_HYDE_CONTEXTS}\n"
    # params_str = params_str + f"READ_QAS_FROM_FILE        = {READ_QAS_FROM_FILE}\n"
    params_str = params_str + f"RESTORE_QAS_WITH_CONTEXTS = {RESTORE_QAS_WITH_CONTEXTS}\n\n"

    params_str = params_str + "--------------------------------------------------------\n"        
    params_str = params_str + f"SAMPLE_SIZE  = {SAMPLE_SIZE}\n"
    params_str = params_str + f"BOOTSTRAPS_N = {BOOTSTRAPS_N}\n\n"

    params_str = params_str + "--------------------------------------------------------\n"
    params_str = params_str + f"LLM_MODEL_NAME_0  = {LLM_MODEL_NAME_0}\n"
    params_str = params_str + f"LLM_MODEL_NAME_1  = {LLM_MODEL_NAME_1}\n"
    params_str = params_str + f"LLM_MODEL_NAME_2  = {LLM_MODEL_NAME_2}\n"
    params_str = params_str + f"LLM_MODEL_NAME_3  = {LLM_MODEL_NAME_3}\n"
    params_str = params_str + f"LLM_MODEL_NAME_4  = {LLM_MODEL_NAME_4}\n"
    params_str = params_str + f"LLM_MODEL_NAME_5  = {LLM_MODEL_NAME_5}\n"
    params_str = params_str + f"CHAT_MODEL_NAME   = {CHAT_MODEL_NAME}\n\n"
    
    params_str = params_str + "--------------------------------------------------------\n"
    params_str = params_str + f"MAX_TOKENS        = {MAX_TOKENS}\n"
    params_str = params_str + f"TEMPERATURE       = {TEMPERATURE}\n"
    params_str = params_str + f"TOP_P             = {TOP_P}\n"
    params_str = params_str + f"FREQUENCY_PENALTY = {FREQUENCY_PENALTY}\n"
    params_str = params_str + f"PRESENCE_PENALTY  = {PRESENCE_PENALTY}\n\n"


    params_str = params_str + "--------------------------------------------------------\n"
    judges = [llm_models[judge_i] for judge_i in judge_llms]
    params_str = params_str + f"Judge LLMS = {judges}\n"
    params_str = params_str + f"JUDGES_SUPPRESS_THRESHOLD   = {JUDGES_SUPPRESS_THRESHOLD}\n\n"

    with open(f"{results_folder_name}/experiments_config_{experiment_name.lower()}.txt", 'w') as f:
        f.write(params_str)

    print("Persisting the experiment config done...")        

In [21]:
def clip_scores(scores_l, clip_perc):
    scores_sorted_l = sorted(scores_l)

    clip_cnt = int(len(scores_sorted_l) * clip_perc / 100)
    print(f"clip_cnt: {clip_cnt}")

    # Now clip both ends by clip_perc percent
    clipped_scores_l = scores_sorted_l[clip_cnt:-clip_cnt] if clip_cnt > 0 else scores_sorted_l
    return clipped_scores_l


In [22]:
def do_bootstrapping(scores_l, results_folder_name, experiment_name, bootstraps_n = BOOTSTRAPS_N):
    
    mu_hats = []
    n = len(scores_l)
    scores_l = clip_scores(scores_l, 2.5)

    for i in range(bootstraps_n):
        if i % 100 == 0:
            print(f"Processing sample {i}...")
        bootstrap_sample = random.choices(scores_l, k=n) # sample with replacement
        mu_hat = np.mean(bootstrap_sample)
        mu_hats.append(mu_hat)

    bootstraps_mean, ci = calculate_mean_confidence_interval(mu_hats)
    plt = generate_histogram(mu_hats, bootstraps_mean, ci, results_folder_name, experiment_name)
    plt.show();
    return bootstraps_mean, ci

In [23]:
def create_results_folder(experiment_name):
    start_timestamp_str = get_timestamp_as_string()
    results_folder_name = f"{RESULTS_ROOT}/{experiment_name}/results_{start_timestamp_str}"
    if not os.path.exists(results_folder_name):
        os.makedirs(results_folder_name, exist_ok=True)
    return results_folder_name

In [24]:
def conduct_experiment(qas, experiment_name, query_prompt_idx, 
                       context_needed=False, hyde_context_needed=False, 
                       suppress_answers=False):

    start_time = time.perf_counter()

    print(f"============= Creating results folder for {experiment_name} =============")
    results_folder_name = create_results_folder(experiment_name)
    
    print(f"============= Calculating scores for {experiment_name} =============")
    print(f"SAMPLE_SIZE: {SAMPLE_SIZE}\n")
    all_preds = calculate_scores(qas, query_prompt_idx, experiment_name, context_needed, 
                                 hyde_context_needed, suppress_answers)

    print(f"============= Persisting results for {experiment_name} =============")
    df = pd.DataFrame(qas)
    persist_results(results_folder_name, experiment_name, df)
    
    print(f"\n============= Bootstrapping for {experiment_name} =============")
    print(f"BOOTSTRAPS_N: {BOOTSTRAPS_N}")

    experiment_mean, experiment_ci = do_bootstrapping(df[f"{experiment_name.lower()}_f1"].dropna().tolist(), results_folder_name, experiment_name, BOOTSTRAPS_N)
    
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    execution_times_entry = {}
    execution_times_entry["experiment_name"] = experiment_name
    execution_times_entry["execution_time"] = execution_time
    execution_times_entry["sample_size"] = SAMPLE_SIZE
    execution_times_entry["bootstrap_n"] = BOOTSTRAPS_N
    execution_times_l.append(execution_times_entry)

    print(f"Will now persist the experiment configuration for {experiment_name}...")
    persist_experiments_config(results_folder_name, experiment_name, experiment_mean, experiment_ci, execution_time)

    return execution_time

##### Creating a sample from df_cas_with_contexts for bootstrapping and bootstrapping with Hyde

In [25]:
df_qas_for_bootstrapping = df_qas_with_contexts[['qid', 'question', 'question_context', 'hyde_based_context']]
df_qas_for_bootstrapping_sample = df_qas_for_bootstrapping.sample(n=SAMPLE_SIZE, replace=True)
ldict_qas_for_boostrapping_sample = df_qas_for_bootstrapping_sample.to_dict(orient='records')

df_qas_for_hyde_bootstrapping = df_qas_for_bootstrapping[df_qas_for_bootstrapping['hyde_based_context'].notna()]
df_qas_for_hyde_bootstrapping_sample = df_qas_for_hyde_bootstrapping.sample(n=SAMPLE_SIZE, replace=True)
ldict_qas_for_hyde_bootstrapping_sample = df_qas_for_hyde_bootstrapping_sample.to_dict(orient='records')

sample_ldicts = [ldict_qas_for_boostrapping_sample, ldict_qas_for_hyde_bootstrapping_sample]

##### Conducting all experiments

In [26]:
# Loop through all experiments and execute them one by one if they are included
for index, row in df_experiments.iterrows():
    if not row["include"]:
        continue

    sample_ldict = sample_ldicts[row["sample_ldicts_idx"]]
    experiment_name = row["name"]
    query_prompt_idx = row["query_prompt_idx"]
    context_needed = row["context_needed"]
    hyde_context_needed = row["hyde_context_needed"]
    suppress_answers = row["suppress_answers"]
    
    print(f"sample_ldict: {sample_ldict}")
    print(f"experiment_name: {experiment_name}")
    print(f"query_prompt_idx: {query_prompt_idx}")
    print(f"context_needed: {context_needed}")
    print(f"hyde_context_needed: {hyde_context_needed}")
    print(f"suppress_answers: {suppress_answers}")
    
    conduct_experiment(sample_ldict, experiment_name, query_prompt_idx,
                       context_needed, hyde_context_needed, suppress_answers)

print(f"\n================== Execution times (in seconds): ==================================\n")
df_execution_times = pd.DataFrame(execution_times_l, 
                                  columns=['experiment_name', 'execution_time', 'sample_size', 'bootstrap_n'])

pd.options.display.float_format = "{:.2f}".format
df_execution_times.head(100)              

print(f"Total execution time in Minutes: {round(sum(df_execution_times['execution_time']) / 60, 2)}")

sample_ldict: [{'qid': '5727f16c3acd2414000df05b', 'question': 'What did Standard & Poor recommend to speed economy recovery?', 'question_context': 'In 2014, economists with the Standard & Poor\'s rating agency concluded that the widening disparity between the U.S.\'s wealthiest citizens and the rest of the nation had slowed its recovery from the 2008-2009 recession and made it more prone to boom-and-bust cycles. To partially remedy the wealth gap and the resulting slow growth, S&P recommended increasing access to education. It estimated that if the average United States worker had completed just one more year of school, it would add an additional $105 billion in growth to the country\'s economy over five years.\n\nCentral Banking economist Raghuram Rajan argues that "systematic economic inequalities, within the United States and around the world, have created deep financial \'fault lines\' that have made [financial] crises more likely to happen than in the past" – the Financial crisis

KeyboardInterrupt: 

In [ ]:
# Reset the display format for floats
pd.reset_option('display.float_format')

In [ ]:
print(f"Total execution time in Minutes: {round(sum(df_execution_times['execution_time']) / 60, 2)}")

In [ ]:
sum([len(cont) for cont in contexts]) / 1204